A small DIY agent that can call a limited subset of python functions.

In [1]:
import openai
import os
import re
from typing import Any, Dict, List, Optional
import pprint
from dotenv import load_dotenv
from datetime import datetime
import json
import requests


pp = pprint.PrettyPrinter()

load_dotenv()
openai.api_key = os.environ["OPENAI_API_KEY"]



In [16]:
def new_hf_chat_generic(inputs):
    API_URL = "https://api-inference.huggingface.co/models/facebook/blenderbot-400M-distill"
    headers = {"Authorization": f"Bearer {os.environ['HUGGINGFACE_API_KEY']}"}
    
    payload = {"inputs": inputs}
    
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

print(new_hf_chat_generic("<human>How are you today?\n<bot>:"))


{'generated_text': " I'm doing well, thank you. How are you? I hope you are as well.", 'conversation': {'generated_responses': [" I'm doing well, thank you. How are you? I hope you are as well."], 'past_user_inputs': ['<human>How are you today?\n<bot>:']}}


In [3]:
# a function that makes a call to the openai API, taking a system message (str) and user message (str)
# and returning a response (str)
# it also calls the maybe_eval_last_message() function to see if the LLM is trying to call a tool (see below)
def start_new_chat_generic(system_message: str, user_message: str, model: str = "gpt-3.5-turbo") -> List[Dict[str, str]]:
    messages = [{"role": "system", "content": system_message},
                {"role": "user", "content": user_message}]
    response = openai.ChatCompletion.create(
              model=model,
              temperature = 0,
              messages=messages)

    response_content = response["choices"][0]["message"]["content"]
    response_role = response["choices"][0]["message"]["role"]
    messages.append({"role": response_role, "content": response_content})

    return messages

In [12]:
# continues a chat returned from start_new_chat() or continue_chat(), 
# taking the current conversation and a new user message
# calls the maybe_eval_last_message() function to see if the LLM is trying to call a tool (see below)
def continue_chat(messages: List[Dict[str, str]], new_user_message: str, model: str = "gpt-3.5-turbo") -> List[Dict[str, str]]:
    messages.append({"role": "user", "content": new_user_message})
    
    response = openai.ChatCompletion.create(
              model=model,
              temperature = 0,
              messages=messages)

    msg_content = response["choices"][0]["message"]["content"]
    msg_role = response["choices"][0]["message"]["role"]
            
    messages.append({"role": msg_role, "content": msg_content})
    messages = maybe_eval_last_message(messages)
    
    return messages

In [4]:
# a class defininng a safe set of callable functions
# (note: also inludes a set of safe functions defined by asteval, including abs(), random.choice(), etc.)
# see example usage below
from asteval import Interpreter


from collections import Counter
from math import log2


class SafeEval:
    def __init__(self):
        self.interpreter = Interpreter()
        self._add_methods()

    def _add_methods(self):
        # Get all methods of the class
        methods = [func for func in dir(self) if callable(getattr(self, func)) and not func.startswith("_")]
        # Add them to the interpreter's symbol table
        for method in methods:
            self.interpreter.symtable[method] = getattr(self, method)

    def evaluate(self, expression: str) -> str:
        return self.interpreter(expression)
    
    def echo(self, x):
        return x
    
    
    
    def sum(self, a, b):
        return a + b

    def product(self, a, b):
        return a * b

    def time(self):
        now = datetime.now()

        # Format the datetime object
        formatted_now = now.strftime("%m/%d/%y %H:%M")

        return formatted_now

    def entropy(self, lst):
        # Compute frequencies
        counter = Counter(lst)

        # Compute probabilities
        probabilities = [count/len(lst) for count in counter.values()]

        # Compute entropy
        return -sum(p * log2(p) for p in probabilities)



# # example usage:
# # Create a SafeEval object
# safe_eval = SafeEval()

# # Test the methods
# #print(safe_eval.evaluate('sum(5, product(2, abs(-3)))'))  # prints 11
# print(safe_eval.evaluate('normalized_entropy(["apple"] * 4 + ["pear"] * 2 + ["peach"] * 10)'))  # prints 11

In [5]:
# runs snippes of python code through the safe evaluator, see example usage below
def replace_eval_tags(text, safe_eval):
    # Regular expression pattern for <eval> tags
    pattern = re.compile(r'<eval>(.*?)</eval>')

    results = []
    # Function to replace each match with its evaluated result
    def replace_with_eval(match):
        code = match.group(1)  # Extract the code string from the match
        results.append(safe_eval.evaluate(code))  # Evaluate the code

    # Replace all <eval> tags in the text
    pattern.sub(replace_with_eval, text)
    return results

# # example usage:
# # Test the function
# safe_eval = SafeEval()
# text = "I need to compute <eval>sum(4, 5)</eval> and <eval>2 + 3</eval>."
# print(replace_eval_tags(text, safe_eval))  # prints [9, 5]



In [7]:
# checks to see if the last message needs interpretation
safe_eval = SafeEval()
def maybe_eval_last_message(messages):
    last_message = messages[-1]["content"]
    computations = replace_eval_tags(last_message, safe_eval)
    if len(computations) > 0:
        # once it tried asking a following question along with a set of computations, and when it got the
        # answer back of just the computation results got confused. 
        result = "RESULT: " + json.dumps(computations) # + "(Warning: other text ignored)"
        return continue_chat(messages, result)
        
    return messages

In [8]:

def new_chat_safeeval_agent(user_message: str, model: str = "gpt-3.5-turbo") -> List[Dict[str, str]]:
    system_prompt = """You are a helpful assistant with the ability to have thoughts, and those 
    thoughts can execute a limited subset of Python code by wrapping it in <eval></eval> tags."""

    first_user_prompt = """You are a helpful assistant with the ability to use tools with 
    the help of your user. These tools allow you to execute a subset of Python code by wrapping it 
    in <eval></eval> tags. 
    
    If you use one or more tool computations, the user will execute them, and provide the results
    as a JSON-formatted list. You should then finish your response based on the computed answer.

    EXAMPLE:
    user: What is the sum of 4 and 5? What is the sum of 2 and 3? What is the product of the prior two answers?
    assistant: I need to compute <eval>sum(4, 5)</eval>, <eval>sum(2, 3)</eval>, and <eval>product(sum(3, 4), sum(2, 3))</eval>
    user: RESULT: [9, 6, 54]
    assistant: The sum of 4 and 5 is 9, the sum of 2 and 3 is 6, and the product of these sums is 54.
    
    EXAMPLE:
    user: A container has 4 apples, 2 pears, and 10 peaches. What is its entropy?
    assistant: I need to compute <eval>normalized_entropy(["apple"] * 4 + ["pear"] * 2 + ["peach"] * 10)</eval>
    user: RESULT: [0.8194483718728035]
    assistant: The entropy is approximately 0.819 <with other information or interpretation as necessary>

    The following functions are available:
    - sum(a, b): returns the sum of numbers a and b
    - product(a, b): returns the product of the numbers a and b
    - entropy(items: List[str]): returns the entropy of a given list of strings
    - echo(x): returns x itself, useful for debugging
    - time(): return the current time in MM/DD/YY HH:MM format

    You should always prefer to use listed tools before computing an answer yourself. Don't repeat yourself.
    
    Confirm by computing the current time.
    """
    #assistant: I have computed the answer. The sum of 4 and 5 is 9, the sum of 2 and 3 is 6, and the product of these is 54.

    messages = start_new_chat_generic(system_prompt, first_user_prompt, model = model)
    messages = maybe_eval_last_message(messages)
    messages = continue_chat(messages, user_message)

    return messages



In [9]:
convo = new_chat_safeeval_agent("What is the entropy of a standard scrabble set?")
pp.pprint(convo)

[{'content': 'You are a helpful assistant with the ability to have thoughts, '
             'and those \n'
             '    thoughts can execute a limited subset of Python code by '
             'wrapping it in <eval></eval> tags.',
  'role': 'system'},
 {'content': 'You are a helpful assistant with the ability to use tools with \n'
             '    the help of your user. These tools allow you to execute a '
             'subset of Python code by wrapping it \n'
             '    in <eval></eval> tags. \n'
             '    \n'
             '    If you use one or more tool computations, the user will '
             'execute them, and provide the results\n'
             '    as a JSON-formatted list. You should then finish your '
             'response based on the computed answer.\n'
             '\n'
             '    EXAMPLE:\n'
             '    user: What is the sum of 4 and 5? What is the sum of 2 and '
             '3? What is the product of the prior two answers?\n'
          